# Process text for entire dataframe
- Remove punctuation, parentheses and convert to lowercase

In [30]:
import nltk
import string
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from collections import Counter
from nltk.stem.porter import *
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

Shamelessly borrowed code from 1) https://www2.cs.duke.edu/courses/spring14/compsci290/assignments/lab02.html  (tf-idf) 2)https://medium.com/mlreview/topic-modeling-with-scikit-learn-e80d33668730 (LDA)

In [31]:
punc1 = '!"#$%\'()*.:;<=>?@[\\]^`{|}~’“”‘–-'
punc2 = ['=', '/', '&', '_', '+', '…', '...']

In [ ]:
# stopwords list 
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
additional_list = ['doin','nn','rd','st','wheres','hows','clean','theyve', 'weve', 'youve', 'de','u', 'yer', 'stuff','cause','mhm', 'mmhm','itit', 'youyou', 'ah', 'ifif', 'there', 'kinda', 'le','ill', 'hell', 'shell', 'whats', 'isnt', 'thats', 'theyve', 'arent', 'couldnt', 'didnt', 'hadnt', 'hasnt', 'werent', 'havent','dont', 'wont', 'cant', 'wouldnt', 'id', 'ive', 'gonna', 'hed', 'shouldnt', 'ii','dr','cuz', 'im','youre', 'hes', 'shes', 'were', 'theyre', 'thethe','theyll', 'youll', 'andand', 'th', 'thatthat', 'sthat', 'wewe','ti','u', 'heh', 'le', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'ya','nd', 'uhh', 's','d','t','by', 'cancer', 'don', 're', 'prostate', 'oh', 'ah', 'ahh', 'm', 'ok', 'okay', 'md', 'like','uh','uhum', 'go', 'got', 'yeah', 'okay', 'yep','uhm', 'umm', 'hum', 'na', 'um', 'legend', 'hmm', 'ah', 'na', 'mm', 'mmm', 'da', 'mmhmm', 'mmmhmm', 'yup', 'hm', 'know', 'would', 'get', 'other', 'huh']
stop_words.extend(additional_list)

In [ ]:
# Functions to help with processing text
def remove_parentheses(txt):
    txt = re.sub('\([^)]*\)\)','', txt) # remove double parentheses 
    txt = re.sub(r'\([^)]*\)', '', txt) # remove single parentheses 
    return txt

def remove_numerical(txt):
    txt = re.sub('[0-9]+', '', txt)
    return txt

def remove_punc(txt):
    for a in punc2:
        txt = txt.replace(a," ")
    for b in punc1:
        txt = txt.replace(b,"")
    return txt

def lowercase(txt):
    txt = txt.lower()
    return txt

In [39]:
df = pd.read_csv('all_transcripts.csv')

In [40]:
df.head()

,Unnamed: 0,patient_id,Convo_1,Convo_2,Dataset,Doctor_1,Doctor_2
0,0,A003,A003 LEGEND: MD2=Physician PT=Patient MD2: So ...,NaN,VA,U,NaN
1,1,A004,A004 LEGEND: MD2=Physician OTH=Study staff-set...,NaN,VA,U,NaN
2,2,A014,A014 LEGEND: MD2=Physician PT=Patient MD2: Tha...,NaN,VA,U,NaN
3,3,A015,A015_CLEAN LEGEND: MD2=Physician PT=Patient MD...,NaN,VA,U,NaN
4,4,A016,A016_CLEAN_LOUD LEGEND: MD2=Physician PT: Pati...,NaN,VA,U,NaN


In [14]:
# clean up each of the convo_1 texts with the processing functions written above
for index, row in df.iterrows():
    if pd.isnull(row['Convo_1']) == False:
        row['Convo_1'] = remove_parentheses(row['Convo_1'])
        row['Convo_1'] = remove_numerical(row['Convo_1'])
        row['Convo_1'] = remove_punc(row['Convo_1'])
        row['Convo_1'] = lowercase(row['Convo_1'])
        df.set_value(index,'Convo_1', row['Convo_1'])

/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


# Tokenizing, removing stopwords, stemming 

## Experiment on one documet

In [304]:
df.head()

,Unnamed: 0,patient_id,Convo_1,Convo_2,Dataset,Doctor_1,Doctor_2
0,0,A003,a legend md physician pt patient md so thank y...,NaN,VA,U,NaN
1,1,A004,a legend md physician oth study staff setting ...,NaN,VA,U,NaN
2,2,A014,a legend md physician pt patient md thank you ...,NaN,VA,U,NaN
3,3,A015,aclean legend md physician pt patient md so it...,NaN,VA,U,NaN
4,4,A016,acleanloud legend md physician pt patient md o...,NaN,VA,U,NaN


In [305]:
convo_0 = df['Convo_1'][0]

In [253]:
tokens = nltk.word_tokenize(convo_0)
tokens

['a',
 'legend',
 'md',
 'physician',
 'pt',
 'patient',
 'md',
 'so',
 'thank',
 'you',
 'for',
 'coming',
 'in',
 'i',
 'know',
 'we',
 'kinda',
 'moved',
 'your',
 'appointment',
 'upbut',
 'next',
 'week',
 'our',
 'clinic',
 'was',
 'kind',
 'of',
 'overbooked',
 'and',
 'we',
 'didn',
 't',
 'have',
 'staff',
 'to',
 'bring',
 'people',
 'in',
 'dr',
 'pt',
 'okay',
 'md',
 'um',
 'so',
 'ah',
 'we',
 'wanted',
 'to',
 'bring',
 'you',
 'in',
 'to',
 'talk',
 'about',
 'your',
 'biopsy',
 'results',
 'um',
 'as',
 'we',
 'kind',
 'of',
 'expected',
 'with',
 'your',
 'psa',
 'being',
 'up',
 'there',
 'was',
 'prostate',
 'cancer',
 'in',
 'the',
 'sample',
 'pt',
 'okay',
 'md',
 'um',
 'the',
 'ah',
 'you',
 'know',
 'the',
 'exam',
 'that',
 'they',
 'did',
 'they',
 'could',
 'feel',
 'a',
 'little',
 'area',
 'that',
 'felt',
 'abnormal',
 'on',
 'the',
 'left',
 'side',
 'of',
 'your',
 'prostate',
 'gland',
 'and',
 'that',
 's',
 'where',
 'the',
 'biopsies',
 'were',
 'p

In [254]:
filtered = [w for w in tokens if not w in stopwords_comprehensive]
filtered

['physician',
 'patient',
 'thank',
 'coming',
 'know',
 'kinda',
 'moved',
 'appointment',
 'upbut',
 'next',
 'week',
 'clinic',
 'kind',
 'overbooked',
 'staff',
 'bring',
 'people',
 'dr',
 'okay',
 'ah',
 'wanted',
 'bring',
 'talk',
 'biopsy',
 'results',
 'kind',
 'expected',
 'psa',
 'prostate',
 'cancer',
 'sample',
 'okay',
 'ah',
 'know',
 'exam',
 'could',
 'feel',
 'little',
 'area',
 'felt',
 'abnormal',
 'left',
 'side',
 'prostate',
 'gland',
 'biopsies',
 'positive',
 'nd',
 'uhh',
 'point',
 'gon',
 'na',
 'talk',
 'kind',
 'grade',
 'prostate',
 'cancer',
 'means',
 'kind',
 'treatment',
 'options',
 'know',
 'probably',
 'given',
 'instruction',
 'least',
 'handouts',
 'kinda',
 'go',
 'know',
 'treat',
 'prostate',
 'cancer',
 'she—if',
 'yet—she',
 'give',
 'educational',
 'today',
 'general',
 'grade',
 'prostate',
 'cancer',
 'looks',
 'microscope',
 'lump',
 'prostate',
 'cancers',
 'risk',
 'categories',
 'call',
 'either',
 'low',
 'risk',
 'intermediate',
 '

In [255]:
# check most common words see if there is anything we should add to stopwords list
count = Counter(filtered)
print(count.most_common(100))

[('cancer', 17), ('risk', 15), ('prostate', 13), ('know', 11), ('would', 9), ('four', 8), ('psa', 7), ('five', 7), ('things', 7), ('spread', 7), ('na', 6), ('treatment', 6), ('low', 6), ('ten', 6), ('like', 6), ('radiation', 6), ('kind', 5), ('okay', 5), ('microscope', 5), ('intermediate', 5), ('aggressive', 5), ('really', 5), ('plus', 5), ('enough', 5), ('active', 5), ('people', 4), ('talk', 4), ('biopsy', 4), ('could', 4), ('point', 4), ('gon', 4), ('grade', 4), ('options', 4), ('common', 4), ('even', 4), ('normal', 4), ('surveillance', 4), ('proton', 4), ('kinda', 3), ('ah', 3), ('little', 3), ('gland', 3), ('go', 3), ('looks', 3), ('high', 3), ('look', 3), ('terms', 3), ('see', 3), ('gleason', 3), ('six', 3), ('one', 3), ('three', 3), ('yeah', 3), ('start', 3), ('best', 3), ('older', 3), ('think', 3), ('worse', 3), ('core', 3), ('va', 3), ('therapy', 3), ('patient', 2), ('moved', 2), ('bring', 2), ('dr', 2), ('wanted', 2), ('results', 2), ('abnormal', 2), ('biopsies', 2), ('means',

In [256]:
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

In [257]:
stemmer = PorterStemmer()
stemmed = stem_tokens(filtered, stemmer)
count = Counter(stemmed)
print(count.most_common(100))

[('cancer', 19), ('risk', 15), ('prostat', 13), ('know', 11), ('psa', 9), ('would', 9), ('four', 8), ('thing', 8), ('look', 7), ('five', 7), ('spread', 7), ('biopsi', 6), ('na', 6), ('grade', 6), ('treatment', 6), ('low', 6), ('ten', 6), ('like', 6), ('radiat', 6), ('kind', 5), ('okay', 5), ('talk', 5), ('option', 5), ('microscop', 5), ('intermedi', 5), ('appear', 5), ('aggress', 5), ('realli', 5), ('plu', 5), ('normal', 5), ('start', 5), ('enough', 5), ('activ', 5), ('core', 5), ('peopl', 4), ('could', 4), ('point', 4), ('gon', 4), ('go', 4), ('categori', 4), ('term', 4), ('common', 4), ('even', 4), ('year', 4), ('surveil', 4), ('proton', 4), ('kinda', 3), ('ah', 3), ('littl', 3), ('gland', 3), ('treat', 3), ('call', 3), ('high', 3), ('see', 3), ('gleason', 3), ('six', 3), ('one', 3), ('three', 3), ('yeah', 3), ('get', 3), ('typic', 3), ('best', 3), ('older', 3), ('think', 3), ('wors', 3), ('va', 3), ('therapi', 3), ('patient', 2), ('come', 2), ('move', 2), ('bring', 2), ('dr', 2), ('

## Apply to process entire conversation 1 column

In [15]:
df_nona = df.dropna(subset=['Convo_1'])
conversations = df_nona['Convo_1']

In [16]:
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
stopwords_comprehensive = stopwords_comprehensive = stopwords.words('english') + added_list

In [17]:
def tokenize(text):
    tokens = nltk.word_tokenize(text)
    #stems = stem_tokens(tokens, stemmer)
    lemmas = lemma_tokens(tokens,lemmatizer)
    #return stems
    return lemmas

In [18]:
def lemma_tokens(tokens, lemmatizer):
    lemmed = []
    for item in tokens:
        lemmed.append(lemmatizer.lemmatize(item))
    return lemmed

In [19]:
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

# TF-

In [398]:
tfidf = TfidfVectorizer(tokenizer=tokenize, max_features=5000, stop_words = stopwords_comprehensive)
tf = tfidf.fit_transform(conversations)

In [399]:
tf_feature_names = tfidf.get_feature_names()

In [401]:
def top_feats_in_doc(Xtr, features, row_id, top_n=25):
    row = np.squeeze(Xtr[row_id].toarray())
    topn_ids = np.argsort(row)[::-1][:top_n] # get indices of biggest tf-idf coefficients 
    top_feats = [(features[i], row[i]) for i in topn_ids] # get corresponding feature/value tuple based off indices
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df

In [402]:
top_feats_in_doc(tf, tf_feature_names, 2, top_n=25)

,feature,tfidf
0,right,0.314188
1,prostate,0.230405
2,cancer,0.195495
3,six,0.189217
4,spinal,0.150540
5,wa,0.147682
6,sphincter,0.145536
7,biopsy,0.141668
8,year,0.140312
9,going,0.137529


TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document)
IDF(t) = log(Total number of documents / Number of documents with term t in it).
To compute, multiply TF(t)*IDF(t) for each term. We get higher tf-idf scores for terms that are really important to certain documents (both in that they appear in few other documents, and appear a lot in a single document). 

## Dictionary of Most Common Words Across Conversations

Finding the most common words can help filter out stopwords

In [25]:
word_counts = Counter()
for conversation in conversations:
    tokens = nltk.word_tokenize(conversation)
    for token in tokens:
        word_counts[token] += 1

In [29]:
word_counts.most_common(400)

[('you', 75168),
 ('the', 70764),
 ('pt', 52826),
 ('that', 52007),
 ('i', 50004),
 ('md', 49193),
 ('and', 48874),
 ('to', 45077),
 ('it', 41444),
 ('a', 39997),
 ('of', 36243),
 ('s', 34364),
 ('so', 33079),
 ('we', 26457),
 ('is', 24915),
 ('okay', 22918),
 ('in', 21077),
 ('know', 20829),
 ('have', 20566),
 ('um', 19896),
 ('yeah', 16816),
 ('t', 15609),
 ('your', 15194),
 ('do', 15084),
 ('for', 13613),
 ('but', 13266),
 ('if', 12970),
 ('prostate', 12965),
 ('they', 12710),
 ('there', 12441),
 ('with', 12360),
 ('or', 12285),
 ('cancer', 12035),
 ('what', 12012),
 ('right', 11458),
 ('be', 11097),
 ('can', 10878),
 ('about', 10813),
 ('not', 10629),
 ('re', 10626),
 ('surgery', 10335),
 ('are', 9900),
 ('on', 9524),
 ('like', 9469),
 ('just', 9424),
 ('this', 9380),
 ('radiation', 9341),
 ('would', 9322),
 ('then', 8418),
 ('don', 8044),
 ('at', 7879),
 ('get', 7783),
 ('no', 7754),
 ('hmm', 7684),
 ('was', 7512),
 ('think', 7012),
 ('uh', 6874),
 ('all', 6766),
 ('well', 6633),


# LDA

In [355]:
# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
# max_df = 0.7, min_df = 3,
c_vectorizer= CountVectorizer(tokenizer=tokenize, max_df = 0.9, max_features=5000, stop_words = stopwords_comprehensive)
cv = c_vectorizer.fit_transform(conversations)
cv_feature_names = c_vectorizer.get_feature_names()

In [406]:
no_topics = 35
# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [404]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [407]:
no_top_words = 10
display_topics(lda, tf_feature_names, no_top_words)

Topic 0:
function free sounding spontaneous syndrome tossup scrotum retired differently literally
Topic 1:
store ton entirely curing information occur culture recurrent friday gain
Topic 2:
nut judgment coordinator booking child limited cryo age convinced friday
Topic 3:
surgery urethral yearly cancer continued prostate leaving injury satellite breathing
Topic 4:
prostate one surgery think hum amount germany intensive sixty achilles
Topic 5:
prostate cancer surgery radiation well right sample think one see
Topic 6:
screw hiding gi resort regimen circle expensive skinny inside hundred
Topic 7:
lowest capsular respiratory nap supervise forecast flash yearly healed stereotactic
Topic 8:
psa control nursing perfect prob probe crummy dentist deliberate test
Topic 9:
prostate cancer pressure provider click data going conference tattoo shit
Topic 10:
deliberate th bill using big decides weaker news steer assessment
Topic 11:
internet straw stroke ekg neurologist catheter adjacent frustrating 